In [1]:
%config IPCompleter.greedy=True
from IPython.display import IFrame

import pandas as pd
import numpy as np
# import string
import math

# import scipy.stats as sts

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn import preprocessing as prep
# import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
# from sklearn import discriminant_analysis as disan
# from sklearn import calibration as calib
# from sklearn import linear_model as lm
# from sklearn import svm
# from sklearn import gaussian_process as gaup
# from sklearn import mixture as mix
# from sklearn import tree
# from sklearn import ensemble as ens

import tensorflow as tf
from tensorflow import keras

# from keras import models as kermdls
# from keras import layers as kerlrs
# from keras import metrics as kmetrics

# from hyperas import optim
# from hyperas.distributions import choice, uniform
# from hyperopt import Trials, STATUS_OK, tpe

# import pickle

# import nilearn as nl
# from nilearn import plotting, image
# from nilearn import datasets
# import nibabel as nb
import h5py

import os

import time
import progressbar
from progress.bar import IncrementalBar
from tqdm import tqdm

from os import environ
from os import listdir
from os.path import isfile, join
from google.cloud import storage
# import gsutil

In [2]:
TEST_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_test'))]
TRAIN_IDS = [map_id.split('.')[0] for map_id in sorted(os.listdir('00_Data/fMRI_train'))]

In [3]:
data = pd.read_csv('00_Data/train_scores_full.csv')
data

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641
3,10007,38.617381,49.197021,65.674285,40.151376,34.096421
4,10008,35.326582,15.769168,65.782269,44.643805,50.448485
...,...,...,...,...,...,...
5872,21654,53.103634,50.951656,62.168022,49.389400,53.020847
5873,21665,38.246437,48.018227,59.522285,45.697098,53.208160
5874,21674,69.414169,58.593918,60.298779,49.865669,47.863167
5875,21693,62.009209,54.272484,60.474388,52.325031,52.989803


In [4]:
data.isnull().sum()

Id              0
age             0
domain1_var1    0
domain1_var2    0
domain2_var1    0
domain2_var2    0
dtype: int64

In [5]:
print('Dataset length: ', len(data.index))

Dataset length:  5877


In [6]:
def inputs_gen(idxs, labels):
    for idx, labs in zip(idxs, labels):
        # FNC inputs
        df_fnc = pd.read_csv('00_Data/fnc_csv/{0}.csv'.format(idx), index_col=0)
        X_fnc = np.array(df_fnc.values).reshape(-1)

        # Loading inputs
        df_loading = pd.read_csv('00_Data/loading_csv/{0}.csv'.format(idx), index_col=0)
        X_loading = np.array(df_loading.values).reshape(-1)

        #MRI inputs
        patient_SM = h5py.File('00_Data/fMRI_train/{0}.mat'.format(idx), mode='r')
        patient_SM = np.array(patient_SM.get('SM_feature'))

        k = 1
        ki_padding = 3

        arr_regions = []
        for i in range(patient_SM.shape[0]):
            sample_map = patient_SM[i,:,:,:]
    #         print(len(arr_regions))
            # padding MRI map
            if k > 1:
                map_shape = sample_map.shape
                shape_pad = ((map_shape[0]//k + 1)*k - map_shape[0],
                             (map_shape[1]//k + 1)*k - map_shape[1],
                             (map_shape[2]//k + 1)*k - map_shape[2])

                npad = ((shape_pad[0]//2, (shape_pad[0]//2 if shape_pad[0]%2==0 else shape_pad[0]//2+1)),    
                        (shape_pad[1]//2, (shape_pad[1]//2 if shape_pad[1]%2==0 else shape_pad[1]//2+1)),    
                        (shape_pad[2]//2, (shape_pad[2]//2 if shape_pad[2]%2==0 else shape_pad[2]//2+1)))

                sample_map_padded = np.pad(sample_map, pad_width=npad, mode='constant', constant_values=0)

                sx = sample_map_padded.shape[0] / k
                sy = sample_map_padded.shape[1] / k
                sz = sample_map_padded.shape[2] / k
                for kz in range(k):
                    for ky in range(k):
                        for kx in range(k):
                            ki_region = sample_map_padded[int(kx*sx): int(kx*sx + sx - 1), 
                                                         int(ky*sy): int(ky*sy + sy - 1), 
                                                         int(kz*sz): int(kz*sz + sz - 1)]
                            #padding i-th region by 3 pixels
                            ki_region_padded = np.pad(ki_region, pad_width=ki_padding, mode='constant', constant_values=0)
                            arr_regions.append(ki_region_padded)
            else:
                map_padded = np.pad(sample_map, pad_width=ki_padding, mode='constant', constant_values=0)
                arr_regions.append(map_padded)
    #             print(map_padded.shape)
        X_mri = np.stack(arr_regions, axis=3)

    #     X = (X_mri, X_fnc, X_loading)
        yield X_mri, X_fnc, X_loading, labs, idx

In [7]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [8]:
def serialize_fullproto(x1, x2, x3, y):
    feat_x1 = _bytes_feature(x1.tostring())
    feat_x2 = _bytes_feature(x2.tostring())
    feat_x3 = _bytes_feature(x3.tostring())
    feat_y = _bytes_feature(y.tostring())

    sample_protobuf = tf.train.Example(features=tf.train.Features(feature={'x1': feat_x1, 'x2': feat_x2, 'x3': feat_x3, 'y': feat_y}))
    return sample_protobuf.SerializeToString()

In [9]:
train, test = model_selection.train_test_split(data, test_size=0.2, shuffle=True, random_state=30)
train, val = model_selection.train_test_split(train, test_size=0.2, shuffle=True, random_state=30)
print(len(train.index))

3760


In [10]:
tfr_train = '00_Data/tfr_train_k1/'
tfr_val = '00_Data/tfr_val_k1/'
tfr_test = '00_Data/tfr_test_k1/'

ops = tf.io.TFRecordOptions(compression_type="GZIP")

In [12]:
with tqdm(total=len(train.index)) as pbar:
    for i, (x1, x2, x3, y, idx) in enumerate(inputs_gen(train['Id'].values, train[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].values)):
        with tf.io.TFRecordWriter('{0}{1}.tfrecord'.format(tfr_train, idx), options=ops) as protowriter:
            sample = serialize_fullproto(x1, x2, x3, y)
            protowriter.write(sample)
            protowriter.flush()
        pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████| 3760/3760 [3:51:28<00:00,  3.69s/it]


In [13]:
with tqdm(total=len(val.index)) as pbar:
    for i, (x1, x2, x3, y, idx) in enumerate(inputs_gen(val['Id'].values, val[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].values)):
        with tf.io.TFRecordWriter('{0}{1}.tfrecord'.format(tfr_val, idx), options=ops) as protowriter:
            sample = serialize_fullproto(x1, x2, x3, y)
            protowriter.write(sample)
            protowriter.flush()
        pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████████| 941/941 [57:02<00:00,  3.64s/it]


In [14]:
with tqdm(total=len(test.index)) as pbar:
    for i, (x1, x2, x3, y, idx) in enumerate(inputs_gen(test['Id'].values, test[['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']].values)):
        with tf.io.TFRecordWriter('{0}{1}.tfrecord'.format(tfr_test, idx), options=ops) as protowriter:
            sample = serialize_fullproto(x1, x2, x3, y)
            protowriter.write(sample)
            protowriter.flush()
        pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████| 1176/1176 [1:11:23<00:00,  3.64s/it]


In [15]:
# Google Cloud Storage
bucketURL = 'https://console.cloud.google.com/storage/browser/trends_2020_tfr'
bucketName = 'trends_2020_tfr'

bucket_train = 'train_k1'
bucket_val = 'val_k1'
bucket_test = 'test_k1'

path_train = '00_Data/tfr_train_k1/'
path_val = '00_Data/tfr_val_k1/'
path_test = '00_Data/tfr_test_k1/'

In [16]:
def upload_files(bucket, bucketFolder, localFolder):
    files = [f for f in listdir(localFolder)]
    with tqdm(total=len(files)) as pbar:
        for file in files:
            localFile = localFolder + file
            blob = bucket.blob(bucketFolder + '/' + file)
            blob.upload_from_filename(localFile)
            pbar.update(1)

In [17]:
storage_client = storage.Client()

In [18]:
bucket = storage_client.get_bucket(bucketName)

In [19]:
upload_files(bucket, bucket_train, path_train)

100%|████████████████████████████████████████████████████████████████████████████| 3760/3760 [5:25:49<00:00,  5.20s/it]


In [20]:
upload_files(bucket, bucket_val, path_val)

100%|██████████████████████████████████████████████████████████████████████████████| 941/941 [1:21:40<00:00,  5.21s/it]


In [21]:
upload_files(bucket, bucket_test, path_test)

100%|████████████████████████████████████████████████████████████████████████████| 1176/1176 [1:42:17<00:00,  5.22s/it]
